In [1]:
import logging
import importlib
importlib.reload(logging) # see https://stackoverflow.com/a/21475297/1469195
log = logging.getLogger()
log.setLevel('INFO')
import sys
import import_ipynb
import actual


logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

importing Jupyter notebook from actual.ipynb
2019-07-24 17:59:43,297 DEBUG : Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [2]:
%%capture
import os
import site


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np


In [3]:
n_examples = 20
n_time_steps = 100
n_width = 11
n_height = 19
inputs = th.ones(n_examples, 1, n_time_steps, n_width * n_height)

In [4]:
device='cuda'

In [5]:
from __future__ import print_function
import torch
import torchvision

In [6]:
traindata = (torch.tensor(actual.images, dtype=torch.float).view(811, 11, 1, 19, 225), torch.tensor(actual.labels))

# trainloader = torch.utils.data.DataLoader(trainset, batch_size=12,
#                                           shuffle=True, num_workers=4)


testdata = (torch.tensor(actual.images_test, dtype=torch.float).view(160, 11, 1, 19, 225), torch.tensor(actual.labels_test))

# testloader = torch.utils.data.DataLoader(testset, batch_size=12,
#                                         shuffle=False, num_workers=4)


In [24]:
n_filters = 15
n_filters_2 = 32
n_filters_3 = 64
n_filters_4 = 128
kernel_size = (1, 21)

# continue: process time dimension several steps,
# then remove it by meaning across time
# -> n_examples x n_conv_out_filters x (width * height)
# -> transform back to:
# n_examples x n_conv_out_filters x width x height
# -> now it is a normal "image" problem

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, n_filters, kernel_size)
        self.conv2 = nn.Conv2d(n_filters, n_filters_2, kernel_size)
        self.conv3 = nn.Conv2d(n_filters_2, n_filters_3, kernel_size)
        # self.conv4 = nn.Conv2d(n_filters_3, n_filters_4, kernel_size)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(n_filters_3 * 209, 500)
        self.fc2 = nn.Linear(500, 100)
        self.fc3 = nn.Linear(100, 20)
        self.fc4 = nn.Linear(20, 4)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (1, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), (1, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (1, 2))
        x = torch.mean(x, -1)
        # print(x.size())
        # x = F.max_pool2d(F.relu(self.conv4(x)), 2)
        # x = x.view(-1, self.num_flat_features(x))
        x = x.view(-1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc4(self.fc3(x)))
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
net.to(device)

Net(
  (conv1): Conv2d(1, 15, kernel_size=(1, 21), stride=(1, 1))
  (conv2): Conv2d(15, 32, kernel_size=(1, 21), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(1, 21), stride=(1, 1))
  (fc1): Linear(in_features=13376, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=20, bias=True)
  (fc4): Linear(in_features=20, out_features=4, bias=True)
)

In [30]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [31]:
d, l = traindata
# print(e) # [19, 1, 11, 225]

In [34]:
i = 0


o = net(d[i].to(device))
print(o)

tensor([0.0092, 0.0864, 0.1272, 0.1833], device='cuda:0',
       grad_fn=<ReluBackward0>)


In [35]:

loss = criterion(o, l[i].to(device))
print(loss)

tensor(0.2007, device='cuda:0', grad_fn=<MseLossBackward>)


In [29]:
# nn.CrossEntropyLoss?
# nn.MSELoss

In [85]:
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i in range(traindata[0].shape[0]):
        # get the inputs; data is a list of [inputs, labels]
        
        inputs, labels = traindata[0][i].to(device), traindata[1][i].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

[1,   200] loss: 0.005
[1,   400] loss: 0.005
[1,   600] loss: 0.008
[1,   800] loss: 0.009
[2,   200] loss: 0.005
[2,   400] loss: 0.005
[2,   600] loss: 0.007
[2,   800] loss: 0.007
[3,   200] loss: 0.005
[3,   400] loss: 0.004
[3,   600] loss: 0.007
[3,   800] loss: 0.007
[4,   200] loss: 0.005
[4,   400] loss: 0.004
[4,   600] loss: 0.005
[4,   800] loss: 0.005
[5,   200] loss: 0.005
[5,   400] loss: 0.003
[5,   600] loss: 0.005
[5,   800] loss: 0.009
[6,   200] loss: 0.005
[6,   400] loss: 0.004
[6,   600] loss: 0.005
[6,   800] loss: 0.004
[7,   200] loss: 0.003
[7,   400] loss: 0.003
[7,   600] loss: 0.003
[7,   800] loss: 0.003
[8,   200] loss: 0.003
[8,   400] loss: 0.004
[8,   600] loss: 0.003
[8,   800] loss: 0.005
[9,   200] loss: 0.003
[9,   400] loss: 0.001
[9,   600] loss: 0.003
[9,   800] loss: 0.005
[10,   200] loss: 0.003
[10,   400] loss: 0.001
[10,   600] loss: 0.002
[10,   800] loss: 0.004
[11,   200] loss: 0.003
[11,   400] loss: 0.001
[11,   600] loss: 0.002
[11,

In [86]:
correct = 0
total = 0
timages, tlabels = testdata
with torch.no_grad():
    for i in range(testdata[0].shape[0]):
        images, labels = timages[i].to(device), tlabels[i].to(device)
        outputs = net(images)
        predicted = torch.max(outputs.data)
        total += labels.size(0)
        correct += (outputs == labels).sum().item()
        # print(outputs, labels)

print('Accuracy of the network on the %d test images: %d %%' % (
    testdata[0].shape[0], 100 * correct / total))


Accuracy of the network on the 160 test images: 59 %


In [71]:
outputs

tensor([0.3460, 0.4153, 0.2221, 0.0000], device='cuda:0')